<a href="https://colab.research.google.com/github/viercame/webscrapping/blob/main/Jumbo_webscraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

JUMBO

In [ ]:
!apt update
!apt install chromium-chromedriver
!pip install selenium

In [ ]:
import sys
import csv
from selenium import webdriver
import time
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import pandas as pd
import asyncio
from bs4 import BeautifulSoup
from google.colab import drive
drive.mount('drive')

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import time
import re

#options = Options()
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
# open it, go to a website, and get results
driver = webdriver.Chrome(options=options)
#driver.get("https://www.tiendasjumbo.co/salud-y-bienestar")

#print(wd.page_source)  # results
# divs = wd.find_elements_by_css_selector('div')

#Define web pages
paginas=['despensa', 
         'panaderia-y-pasteleria', 
         'lacteos-huevos-y-refrigerados', 
         'charcuteria',
         'productos-congelados',
         'dulces-y-postres',
         'pasabocas',
         'cuidado-de-ropa-y-calzado',
         'aseo-de-hogar',
         'cuidado-personal',
         'belleza',
         'limpieza-de-cocina']



# default number of scroll downs
from_page = 1
num_page = 10
fecha = '2021-10-29'
#dfjumbo = pd.DataFrame(columns = ['fecha', 'categoria', 'product-item__name', 'discount-percent', 'product-prices__value--best-price', 'url', 'product-item__brand'])
dfjumbo=pd.DataFrame()
dfjumbofinal=pd.DataFrame()

indicedfjumbo = 0
'''
async def run(x):
    limit = asyncio.Semaphore(10)
    from_page = 1
    number_page = 3
    for i in range(from_page, number_page):
        url = f'https://www.tiendasjumbo.co/supermercado/' + x
        await scraper(url, limit)
'''
#Funcion para convertir soup to dataframe. Aplica para esquema de Jumbo
def get_jumbo(soup, clase, atributo):
    results=[]
    
    #print(clase)
    for element in soup.findAll(attrs={'class': clase}):
        print(element)
        if atributo=='text':
            name =element.get_text()
        else:
            name = element.get_attribute_list(atributo)[0]
        #print(name)
        results.append(name)
    return results

for x in paginas:
    print(x)
    #Defin URL
    url = 'https://www.tiendasjumbo.co/supermercado/' + x
    driver.get(url)
    
    pre_scroll_height = driver.execute_script('return document.body.scrollHeight;')
    #print(pre_scroll_height)
    run_time, max_run_time = 0, 5

    while True:
        iteration_start = time.time()
        # Scroll webpage, the 100 allows for a more 'aggressive' scroll
        try:
            driver.execute_script('window.scrollTo(0, 100*document.body.scrollHeight);')
            post_scroll_height = driver.execute_script('return document.body.scrollHeight;')
        except TimeoutException as ex:
            break

        #print(post_scroll_height)
        scrolled = post_scroll_height != pre_scroll_height
        #print(scrolled)
        timed_out = run_time >= max_run_time

        if scrolled:
            run_time = 0
            pre_scroll_height = post_scroll_height
        elif not scrolled and not timed_out:
            run_time += time.time() - iteration_start
        elif not scrolled and timed_out:
            break

    driver.quit

    #Read lxml in BS4
    soup=BeautifulSoup(driver.page_source, 'lxml')
    #print('Conversion a BS4 exitosa')

    #Almacena la informacion de soup en el dataframe
    temp={}
    #Se busca el div que agrega toda la informacion de cada producto.
    for element in soup.findAll('div', class_= re.compile('product-item product-item')):
        if element.find(attrs={'class':'product-item__no-stock'}) is None:
            temp['productName'] = element.find(attrs={'class':'product-item__name'}).get_attribute_list('title')[0]
            temp['link']= element.find(attrs={'class':'product-item__name'}).get_attribute_list('href')[0]
            temp['descuento'] = element.find(attrs={'class':'discount-percent'}).get_attribute_list('data-discount')[0]
            temp['lowPrice'] = element.find(attrs={'class':'product-prices__value product-prices__value--best-price'}).get_text()
            temp['brand'] = element.find(attrs={'class':'product-item__brand'}).get_text()
            dfjumbo = dfjumbo.append(temp, ignore_index=True)
    dfjumbo['categoria1'] = x
    
    dfjumbofinal = dfjumbofinal.append(dfjumbo, ignore_index=True)
    dfjumbo=pd.DataFrame()

#Asigna fecha de consulta
dfjumbofinal['fecha'] = fecha
dfjumbofinal['almacen'] = 'JUMBO'

nombrearchivo = fecha + ' jumbo.csv'
dfjumbofinal.to_csv("/content/drive/MyDrive/Vicente/Python/Mercado/" + nombrearchivo, index=False)

'''#Codigo para obtener data desde Selenium
    a=driver.find_elements(By.CLASS_NAME, 'product-item__name')
    b=driver.find_elements(By.CLASS_NAME, 'discount-percent')
    c=driver.find_elements(By.CLASS_NAME, 'product-prices__value--best-price')
    #d=driver.find_elements(By.CLASS_NAME, "product-specification__technical-data-field")
    e=driver.find_elements(By.CLASS_NAME, 'product-item__brand')

    for element in range(len(a)):
        print("elemento " + str(element))
        titulo =  a[element].get_attribute('title')
        descuento = b[element].get_attribute('data-discount')
        precio = c[element].text
        url = a[element].get_attribute('href')
        marca = e[element].text
        dfjumbo.loc[indicedfjumbo + element] = [fecha, x, titulo, descuento, precio, url, marca]
    
    indicedfjumbo = indicedfjumbo + element + 1 
    #print(indicedfjumbo)
'''

Drive already mounted at drive; to attempt to forcibly remount, call drive.mount("drive", force_remount=True).
despensa
panaderia-y-pasteleria
lacteos-huevos-y-refrigerados
charcuteria
productos-congelados
dulces-y-postres
pasabocas
cuidado-de-ropa-y-calzado
aseo-de-hogar
cuidado-personal
belleza
limpieza-de-cocina


'#Codigo para obtener data desde Selenium\n    a=driver.find_elements(By.CLASS_NAME, \'product-item__name\')\n    b=driver.find_elements(By.CLASS_NAME, \'discount-percent\')\n    c=driver.find_elements(By.CLASS_NAME, \'product-prices__value--best-price\')\n    #d=driver.find_elements(By.CLASS_NAME, "product-specification__technical-data-field")\n    e=driver.find_elements(By.CLASS_NAME, \'product-item__brand\')\n\n    for element in range(len(a)):\n        print("elemento " + str(element))\n        titulo =  a[element].get_attribute(\'title\')\n        descuento = b[element].get_attribute(\'data-discount\')\n        precio = c[element].text\n        url = a[element].get_attribute(\'href\')\n        marca = e[element].text\n        dfjumbo.loc[indicedfjumbo + element] = [fecha, x, titulo, descuento, precio, url, marca]\n    \n    indicedfjumbo = indicedfjumbo + element + 1 \n    #print(indicedfjumbo)\n'

In [ ]:
from google.colab import data_table
data_table.enable_dataframe_formatter()

In [ ]:
dfjumbofinal

,brand,descuento,link,lowPrice,productName,categoria1,fecha,almacen
0,DIANA,0,https://www.tiendasjumbo.co/frijol-diana-bola-...,$5.390,Frijol Diana bola roja x 500g,despensa,2021-10-29,JUMBO
1,REFISAL,0,https://www.tiendasjumbo.co/bolsa-sal-refisal-...,$1.790,Sal Refisal Bolsa x 1000 g,despensa,2021-10-29,JUMBO
2,CORONA,0,https://www.tiendasjumbo.co/chocolate-resellab...,$6.190,Chocolate Resellable Corona x 500G,despensa,2021-10-29,JUMBO
3,DIANA,"4,65 %",https://www.tiendasjumbo.co/arroz-diana-x-5-kg/p,$12.291,Arroz Diana x 5 Kg,despensa,2021-10-29,JUMBO
4,DORIA,"30,0 %",https://www.tiendasjumbo.co/pasta-clasica-spag...,$3.353,Pasta Clásica Spaghetti Doria x 1000 g.,despensa,2021-10-29,JUMBO
...,...,...,...,...,...,...,...,...
5421,TASK,"25,01 %",https://www.tiendasjumbo.co/guantes-task-cocin...,$5.992,Guantes Task cocina talla M,limpieza-de-cocina,2021-10-29,JUMBO
5422,SALVO,"25,01 %",https://www.tiendasjumbo.co/salvo-limon-deterg...,$2.992,Lavaloza Líquido Salvo Limón x300mL,limpieza-de-cocina,2021-10-29,JUMBO
5423,AXION,"25,01 %",https://www.tiendasjumbo.co/lavaplatos-axion-l...,$3.667,Lavaloza Axion Limón Líquido botella x400ml,limpieza-de-cocina,2021-10-29,JUMBO
5424,FAMILIA,0,https://www.tiendasjumbo.co/servilletas-famili...,$3.590,Servilletas Familia Delux x 40 und,limpieza-de-cocina,2021-10-29,JUMBO


In [ ]:
dfjumbofinal.info()

In [ ]:
#para ver los atributos que tiene cada driver.
'''
attrs=[]
for attr in e[2].get_property('attributes'):
    attrs.append([attr['name'], attr['value']])
print(attrs)

'''